# SSIM - Structural Similarity Index Metric 

#### Source: https://ffmpeg.org/ffmpeg-filters.html#ssim

See [1](https://ieeexplore.ieee.org/document/4775883) and [2](https://ieeexplore.ieee.org/document/1284395).


 - [1] Zhou Wang; Bovik, A.C., "Mean squared error: Love it or leave it? A new look at Signal Fidelity Measures", Signal Processing Magazine, IEEE, vol. 26, no. 1, pp. 98-117, Jan. 2009.

 - [2] Z. Wang, A. C. Bovik, H. R. Sheikh and E. P. Simoncelli, "Image quality assessment: From error visibility to structural similarity", IEEE Transactions on Image Processing, vol. 13, no. 4, pp. 600-612, Apr. 2004.

## Example:

This ffmpeg filter takes in input two input videos, the first input is the distorted and the second input is used as a "reference" video for computing the PSNR.
As the distorted video has a different resolution, the first step is to upscale:

In [ ]:
!ffmpeg -hide_banner -y -r 25 -i videos/dist/x264/bbb_640x360_CRF40.mp4 -vf scale=1920x1080 -c:v libx264 -crf 0 /tmp/dist.mp4

Both videos must have the same resolution and pixel format for this filter to work correctly. Also it assumes that both inputs have the same number of frames, which are compared one by one. The filter computes the SSIM of each frame.

In [ ]:
! mkdir -p results/ssim && \
ffmpeg -hide_banner -r 25 -i /tmp/dist.mp4 -r 25 -i videos/ref/bbb_1920x1080.mp4 -lavfi \
    "[0:v]settb=AVTB,setpts=PTS-STARTPTS[distorted];\
     [1:v]settb=AVTB,setpts=PTS-STARTPTS[reference];\
     [distorted][reference]ssim=f=results/ssim/bbb_ssim.txt" -f null -

### Graphical representation.

In [ ]:
import pandas as pd 
import re
import plotly.express as px

# This is the file with the computed SSIM (adapt to your needs)
file='results/ssim/bbb_ssim.txt'

path_file = open(file, 'r')

lines = path_file.readlines()

ssim_frames=[]

for line in lines:
    float_frame=re.sub("[ :]", ",", line)
    lts_frame=float_frame.split(',')
    ssim_frames.append([float(lts_frame[3]),float(lts_frame[5]),
                        float(lts_frame[7]),float(lts_frame[9])])

df=pd.DataFrame(ssim_frames, columns=['Y','U','V','SSIM'])

fig = px.line(df,x=df.index, y='SSIM',
              title='SSIM by frame',
              labels={"index": "Frame"}, 
              template='simple_white',
              markers=True)

fig.add_hline(y=df.SSIM.mean(), line_width=2, line_dash='dash')

fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True)

fig.show()

This second plot shows the SSIM and the SSIM per component (Y, U, V)

In [ ]:
import pandas as pd 
import re
import plotly.graph_objects as go


# This is the file with the computed SSIM (adapt to your needs)
file='results/ssim/bbb_ssim.txt'

path_file = open(file, 'r')
lines = path_file.readlines()

ssim_frames=[]

for line in lines:
    float_frame=re.sub("[ :]", ",", line)
    lts_frame=float_frame.split(',')
    ssim_frames.append([float(lts_frame[3]),float(lts_frame[5]),
                        float(lts_frame[7]),float(lts_frame[9])])

df=pd.DataFrame(ssim_frames, columns=['Y','U','V','SSIM'])

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df.SSIM,
                    mode='lines+markers',
                    name='SSIM',
                         line=dict(color='#3E4680')))
fig.add_trace(go.Scatter(x=df.index, y=df.Y,
                    mode='lines+markers',
                    name='Y',
                         line=dict(color='#FFD0A1')))
fig.add_trace(go.Scatter(x=df.index, y=df.U,
                    mode='lines+markers',
                    name='U',
                          line=dict(color='#C56E90')))
fig.add_trace(go.Scatter(x=df.index, y=df.V,
                    mode='lines+markers',
                    name='V',
                          line=dict(color='#80BFA0')))

fig.update_layout(
    title={'text':'<b>SSIM by frame</b>',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center'},
    showlegend=True,
    template='simple_white',
    xaxis_title='Frame',yaxis_title='SSIM')

fig.show()